In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.cluster import KMeans
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
sns.set(context = "notebook",palette = "Spectral", style = "darkgrid", font_scale = 1.5, color_codes = True)


In [ ]:
def find_percentage_from_dict(dictionary_ref):
    total = 0
    dictionary = dictionary_ref.copy()
    for key in dictionary:
        total += dictionary[key]
    for key in dictionary:
        dictionary[key] = dictionary[key]*100 / total
    return dictionary

def make_position_dict_from_soccer_dataframe(df,label_list ,chosen_label):
    position_dict_by_label = {}
    for i in range(df.shape[0]):
        for position in df['player_positions'][i].replace(" ","").split(','):
            if((position not in position_dict_by_label)):
                position_dict_by_label[position] = 0
                
    for i in range(raw_df.shape[0]):
        for position in df['player_positions'][i].replace(" ","").split(','):
            if(chosen_label == None or label_list[i] == chosen_label):
                position_dict_by_label[position] = position_dict_by_label[position] + 1
                
    return position_dict_by_label

def find_percentage_from_dataframe_and_labels(df,label_list,chosen_label):
    return find_percentage_from_dict(make_position_dict_from_soccer_dataframe(df,label_list,chosen_label))

def merge_right_left_positions(dictionary_ref):
    dictionary = dictionary_ref.copy()
    if('RW' in dictionary and 'LW' in dictionary):
        dictionary['SW'] = dictionary.pop('RW') + dictionary.pop('LW')
    if('RB' in dictionary and 'LB' in dictionary):
        dictionary['SB'] = dictionary.pop('RB') + dictionary.pop('LB')
    if('RM' in dictionary and 'LM' in dictionary):
        dictionary['SM'] = dictionary.pop('RM') + dictionary.pop('LM')
    if('RWB' in dictionary and 'LWB' in dictionary):
        dictionary['SWB'] = dictionary.pop('RWB') + dictionary.pop('LWB')
    return dictionary

In [ ]:
df = pd.read_csv('../input/football-player/players_20_numeric_normalized.csv',index_col = 0)
df.isna().sum()
df.drop_duplicates(inplace = True)
print(df.columns)

raw_df = pd.read_csv("../input/fifa-dataset/players_20.csv/players_20.csv")

In [ ]:
# from sklearn.cluster import KMeans
# wcss = []
# for i in range(1,11):
#     kmeans = KMeans(n_clusters = i, init = 'k-means++',random_state = 42)
#     kmeans.fit(df)
#     wcss.append(kmeans.inertia_)
    
# plt.figure(figsize=(10,5))
# sns.lineplot(range(1,11), wcss, marker='o',color = 'red')
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

In [ ]:
#Choose k that has reasonable wcss decrement

k = 4
kmeans = KMeans(n_clusters = k, init = 'k-means++', random_state = 100)
labels = kmeans.fit_predict(df)
print(sk.metrics.calinski_harabasz_score(df,labels))
print(sk.metrics.davies_bouldin_score(df,labels))
print(sk.metrics.silhouette_score(df,labels))

In [ ]:
find_percentage_from_dataframe_and_labels(raw_df,labels,0)

In [ ]:
find_percentage_from_dataframe_and_labels(raw_df,labels,1)

In [ ]:
myDictionary = merge_right_left_positions(find_percentage_from_dataframe_and_labels(raw_df,labels,0))
plt.figure(figsize=(30,10))
plt.bar(myDictionary.keys(), myDictionary.values(), color='g')
plt.show()

In [ ]:
myDictionary = merge_right_left_positions(find_percentage_from_dataframe_and_labels(raw_df,labels,1))
plt.figure(figsize=(30,10))
plt.bar(myDictionary.keys(), myDictionary.values(), color='g')
plt.show()

In [ ]:
myDictionary = merge_right_left_positions(find_percentage_from_dataframe_and_labels(raw_df,labels,2))
plt.figure(figsize=(30,10))
plt.bar(myDictionary.keys(), myDictionary.values(), color='g')
plt.show()

In [ ]:
myDictionary = merge_right_left_positions(find_percentage_from_dataframe_and_labels(raw_df,labels,3))
plt.figure(figsize=(30,10))
plt.bar(myDictionary.keys(), myDictionary.values(), color='g')
plt.show()

In [ ]:
myDictionary = merge_right_left_positions(find_percentage_from_dataframe_and_labels(raw_df,labels,None))
plt.figure(figsize=(30,10))
plt.bar(myDictionary.keys(), myDictionary.values(), color='g')
plt.show()

In [ ]:
def plot_stacked_bar(data, series_labels, category_labels=None, 
                     show_values=False, value_format="{}", y_label=None, 
                     colors=None, grid=True, reverse=False):
    """Plots a stacked bar chart with the data and labels provided.

    Keyword arguments:
    data            -- 2-dimensional numpy array or nested list
                       containing data for each series in rows
    series_labels   -- list of series labels (these appear in
                       the legend)
    category_labels -- list of category labels (these appear
                       on the x-axis)
    show_values     -- If True then numeric value labels will 
                       be shown on each bar
    value_format    -- Format string for numeric value labels
                       (default is "{}")
    y_label         -- Label for y-axis (str)
    colors          -- List of color labels
    grid            -- If True display grid
    reverse         -- If True reverse the order that the
                       series are displayed (left-to-right
                       or right-to-left)
    """

    ny = len(data[0])
    ind = list(range(ny))

    axes = []
    cum_size = np.zeros(ny)

    data = np.array(data)

    if reverse:
        data = np.flip(data, axis=1)
        category_labels = reversed(category_labels)

    for i, row_data in enumerate(data):
        color = colors[i] if colors is not None else None
        axes.append(plt.bar(ind, row_data, bottom=cum_size, 
                            label=series_labels[i], color=color))
        cum_size += row_data

    if category_labels:
        plt.xticks(ind, category_labels)

    if y_label:
        plt.ylabel(y_label)

    plt.legend()

    if grid:
        plt.grid()

    if show_values:
        for axis in axes:
            for bar in axis:
                w, h = bar.get_width(), bar.get_height()
                plt.text(bar.get_x() + w/2, bar.get_y() + h/2, 
                         value_format.format(h), ha="center", 
                         va="center")

In [ ]:
full_position_dict = make_position_dict_from_soccer_dataframe(raw_df,labels,None)
position_dicts_by_label = []
for i in range(k):
    position_dicts_by_label.append(make_position_dict_from_soccer_dataframe(raw_df,labels,i))
position_dicts_by_label

data = []
for i in range(k):
    position_percentage_list = []
    for key in full_position_dict.keys(): 
        position_percentage_list.append(position_dicts_by_label[i][key]/full_position_dict[key])
    data.append(position_percentage_list)  

In [ ]:
myDictionary = make_position_dict_from_soccer_dataframe(raw_df,labels,None)

plt.figure(figsize=(25, 20))
plt.rcParams.update({'font.size': 15})

series_labels = ['Cluster1', 'Cluster2','Cluster3','Cluster4']
category_labels = [position for position in myDictionary.keys()]
plot_stacked_bar(
    data, 
    series_labels, 
    category_labels=category_labels, 
    show_values=True, 
    value_format="{:.2f}",
    colors=['tab:orange', 'tab:green','tab:blue','tab:brown'],
    y_label="Percentages"
)
plt.savefig("Stacked-bar-chart-kmeans")
plt.show()